In [27]:
import numpy as np
from cvxopt.base import matrix, spmatrix
from cvxopt.modeling import op
import os


def read_mps_preprocess(filepath):
    os.system("./emps %s >> tmp.mps" % filepath)
    problem = op()
    problem.fromfile('tmp.mps')
    mat_form = problem._inmatrixform(format='dense')
    format = 'dense'
    assert mat_form
    lp, vmap, mmap = mat_form

    variables = lp.variables()
    x = variables[0]
    c = lp.objective._linear._coeff[x]
    inequalities = lp._inequalities
    G = inequalities[0]._f._linear._coeff[x]
    h = -inequalities[0]._f._constant
    equalities = lp._equalities
    A, b = None, None
    if equalities:
        A = equalities[0]._f._linear._coeff[x]
        b = -equalities[0]._f._constant
    elif format == 'dense':
        A = matrix(0.0, (0,len(x)))
        b = matrix(0.0, (0,1))
    else:
        A = spmatrix(0.0, [], [],  (0,len(x)))  # CRITICAL
        b = matrix(0.0, (0,1))

    c = np.array(c).flatten()
    G = np.array(G)
    h = np.array(h).flatten()
    A = np.array(A)
    b = np.array(b).flatten()

    return c, G, h, A, b

In [14]:
import requests

In [15]:
resp = requests.get('http://www.netlib.org/lp/data/25fv47')

In [16]:
text = resp.text

In [17]:
with open("25fv47",'w') as f:
    f.write(text)

In [28]:
c,G,h,A,b = read_mps_preprocess("cre-a")

removing redundant constraint 'F1X.0'


In [29]:
n = c.shape[0]

In [30]:
import docplex

from docplex.mp.model import Model

m = Model(name='aaa')
x = m.continuous_var_list(n)

for l,r in zip(A,b):
    m.add_constraint( m.dot(x,l) == r)
    
for l,r in zip(G,h):
    m.add_constraint( m.dot(x,l) <= r)
    
m.minimize(m.dot(x,c))

import time
t = time.time()
m.solve()
print ("time used:",time.time() -t )

time used: 0.08469390869140625


In [31]:
m.print_solution()

objective: 5501.846
  _x1=53.139
  _x4=34.226
  _x6=7.350
  _x8=17.315
  _x9=2081.834
  _x11=62.500
  _x12=1294.227
  _x14=373.812
  _x15=132.188
  _x16=146.542
  _x17=124.857
  _x18=39.748
  _x19=17.765
  _x21=246.088
  _x22=2.809
  _x23=326.743
  _x24=178.256
  _x25=122.669
  _x26=335.523
  _x27=132.585
  _x28=272.706
  _x29=30.708
  _x30=272.000
  _x33=102.767
  _x34=163.701
  _x35=14.218
  _x36=903.410
  _x37=89.350
  _x38=3.780
  _x39=104.307
  _x40=95.693
  _x42=8.885
  _x44=70.115
  _x45=148.000
  _x47=146.000
  _x52=41.709
  _x53=205.291
  _x55=36.414
  _x68=12.181
  _x70=144.876
  _x71=101.901
  _x72=144.198
  _x73=2.777
  _x74=33.820
  _x75=17.137
  _x76=2.499
  _x77=0.904
  _x88=25.408
  _x94=0.165
  _x101=26.366
  _x104=45.802
  _x107=13.320
  _x109=7.000
  _x110=8.611
  _x111=27.358
  _x112=4.217
  _x113=9.279
  _x114=6.643
  _x115=6.114
  _x117=64.779
  _x121=2.872
  _x122=1.425
  _x124=5.634
  _x125=13.063
  _x126=45.000
  _x127=42.696
  _x128=1.683
  _x129=6.665
  _x130